In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("meta_train.csv")

In [3]:
X = df.drop("label", axis=1)
y = df["label"]

In [25]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [26]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [5]:
scaler_meta=StandardScaler()
X=scaler_meta.fit_transform(X)

In [29]:
def objective(trial):
    #Hyperparameter search space
    use_two_layers = trial.suggest_categorical("use_two_layers", [True, False])
    if use_two_layers:
        hidden_layer_sizes = (
            trial.suggest_int("layer1", 32, 256),
            trial.suggest_int("layer2", 16, 128),
        )
    else:
        hidden_layer_sizes = (
            trial.suggest_int("single_layer", 32, 256),
        )

    activation = trial.suggest_categorical("activation", ["relu", "tanh", "logistic"])
    solver = trial.suggest_categorical("solver", ["adam", "lbfgs"])
    alpha = trial.suggest_float("alpha", 1e-5, 1e-2, log=True)
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-4, 1e-2, log=True)

    #Model inside a pipeline with StandardScaler
    clf = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate_init=learning_rate_init,
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=10,
    validation_fraction=0.1,
    random_state=42
)

    

    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy", n_jobs=-1)

    return np.mean(scores)

In [30]:
print(X.shape)

(411230, 30)


In [31]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2025-11-06 23:30:08,502] A new study created in memory with name: no-name-164d1fd3-4cb2-43d9-9e3b-ee726be7f861


[I 2025-11-06 23:31:40,679] Trial 0 finished with value: 0.9999951365534212 and parameters: {'use_two_layers': True, 'layer1': 186, 'layer2': 80, 'activation': 'logistic', 'solver': 'adam', 'alpha': 1.901193007780401e-05, 'learning_rate_init': 0.0003839605887466065}. Best is trial 0 with value: 0.9999951365534212.
[I 2025-11-06 23:32:21,961] Trial 1 finished with value: 0.9999902730891027 and parameters: {'use_two_layers': False, 'single_layer': 242, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0003220677080371485, 'learning_rate_init': 0.008376608131899945}. Best is trial 0 with value: 0.9999951365534212.
[I 2025-11-06 23:34:14,918] Trial 2 finished with value: 0.9999951365534212 and parameters: {'use_two_layers': True, 'layer1': 243, 'layer2': 26, 'activation': 'logistic', 'solver': 'lbfgs', 'alpha': 0.002211790774984368, 'learning_rate_init': 0.0014849340413027295}. Best is trial 0 with value: 0.9999951365534212.
[I 2025-11-06 23:35:43,171] Trial 3 finished with value: 0.99999

In [7]:
from sklearn.neural_network import MLPClassifier

In [6]:
import joblib

In [8]:
joblib.dump(scaler_meta,"scaler_meta.pkl")

['scaler_meta.pkl']

In [34]:
meta_best=study.best_params

In [35]:
print(meta_best)

{'use_two_layers': True, 'layer1': 148, 'layer2': 70, 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.0011296960044143618, 'learning_rate_init': 0.0002343821552649794}


In [36]:
del meta_best["use_two_layers"]

In [45]:
del meta_best["layer1"]

In [46]:
del meta_best["layer2"]

In [42]:
meta_best["hidden_layer_sizes"]=(148,70)

In [47]:
joblib.dump(meta_best,"meta_best.pkl")

['meta_best.pkl']

In [13]:
meta_best=joblib.load("meta_best.pkl")

In [14]:
meta_model=MLPClassifier(**meta_best,random_state=42)
meta_model.fit(X,y)

MLPClassifier(activation='logistic', alpha=0.0011296960044143618,
              hidden_layer_sizes=(148, 70),
              learning_rate_init=0.0002343821552649794, random_state=42)

In [15]:
joblib.dump(meta_model,"meta_model.pkl")

['meta_model.pkl']